<a href="https://colab.research.google.com/github/VerebicsPeter/CodeSim/blob/main/model/model_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Similarity with Finetuning

## Dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint as pp
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
# Hugging Face Transformers (fro CodeBERT etc.)
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
# Libraries for logging
from tqdm.auto import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

Device: cuda
Tesla T4


## Dataset Access

In [3]:
from google.colab import userdata
paired_dataset_url = f"https://drive.google.com/uc?export=download&id={userdata.get('paired_dataset_url')}"

## Dataset

In [4]:
COLUMNS = [
    'pid',    # CodeNet problem ID
    'sid_x',  # CodeNet solution ID of 'src_x'
    'sid_y',  # CodeNet solution ID of 'src_y'
    'src_x',  # CodeNet solution code of 'sid_x'
    'src_y',  # CodeNet solution code of 'sid_y'
    'label',  # Label indicating if 'src_x' and 'src_y' both solve 'pid'
]

In [5]:
df = pd.read_csv(paired_dataset_url, header=0, names=COLUMNS)
df = df.drop(columns=['pid', 'sid_x', 'sid_y'])
df.head()

,src_x,src_y,label
0,##C - Step(TLE)\nN = int(input())\nA = list(ma...,"N=int(input())\nA=list(map(int,input().split()...",1
1,S = input()\nINF = 10 ** 9 + 7\nlength = len(S...,s = input()\nn = len(s)\n\nMOD = 10**9+7\ndp =...,1
2,import sys\nfrom bisect import bisect_right as...,'''\n研究室PCでの解答\n'''\nimport math\n#import nump...,1
3,"numb=input()\ninputs = list(map(int,input().sp...","N = int(input())\na = list(map(int, input().sp...",1
4,"import sys, re\nfrom math import ceil, sqrt, h...","from itertools import permutations\nn,m=map(in...",1


In [6]:
# Code datasets
# TODO: Filter out sequences that don't fit the model's sequence length

class CodePairDataset(Dataset):
    def __init__(self, codes_a, codes_b, labels, tokenizer):
        super().__init__()
        assert len(codes_a) == len(codes_b) == len(labels), "Length MUST match!"
        self.codes_a = codes_a
        self.codes_b = codes_b
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        code_a = self.codes_a[idx]
        code_b = self.codes_b[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            code_a, code_b,
            padding='max_length',  # Pad to max_length
            max_length=self.tokenizer.model_max_length,
            truncation=True,       # Truncate to max_length
            return_tensors='pt'    # Return torch.Tensor objects
        )
        # Remove batch dimension
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        return encoding, label

    def __len__(self):
        return len(self.labels)

    @classmethod
    def from_csv_data(cls, path: str, tokenizer):
        df = pd.read_csv(paired_dataset_url, header=0, names=COLUMNS)
        df = df.drop(columns=['pid', 'sid_x', 'sid_y'])
        srcs_x = df['src_x'].to_list()
        srcs_y = df['src_y'].to_list()
        labels = df['label'].to_list()
        return cls(srcs_x, srcs_y, labels, tokenizer)

## Model

In [7]:
# Model for finetuning

class CodeSimilarityClassifier(nn.Module):
    def __init__(self,
        bert,  # BERT based model instance
        freeze_bert=False,
        dropout_rate=0.2,
    ):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(dropout_rate)
        self.cls = nn.Linear(self.bert.config.hidden_size, 1)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, bert_input) -> torch.Tensor:
        with torch.device(device):
            bert_output = self.bert(**bert_input)
            pooler_output = bert_output.pooler_output
            pooler_output = self.drop(pooler_output)
            logits = self.cls(pooler_output)
            return logits

## Training

In [8]:
def train_one_epoch(
    model, device,
    loss_func,
    scaler,
    optimizer,
    scheduler,
    dataloader,
    iters_to_accumulate,
    # for logging
    print_every=10
):
    running_loss = 0.0
    model.train()

    num_iter = len(dataloader)
    for iter, (encoding, labels) in enumerate(tqdm(train_loader)):
        # Converting to cuda tensors
        for k, v in encoding.items(): encoding[k] = v.to(device)
        labels = labels.to(device)

        # Obtaining the logits from the model
        # Enables autocasting for the forward pass (model + loss)
        with autocast('cuda'):
            # Obtaining the logits from the model
            logits = model(encoding)
            # Computing loss
            loss = loss_func(logits.squeeze(-1), labels.float())
            # Normalize the loss because it is averaged
            loss = loss / iters_to_accumulate

        # Backpropagating the gradients
        # Scales loss. (calls backward() on scaled loss to create scaled gradients)
        scaler.scale(loss).backward()

        if (iter + 1) % iters_to_accumulate == 0:
            # Optimization step
            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            # If these gradients do not contain infs or NaNs, opti.step() is then called,
            # otherwise, opti.step() is skipped.
            scaler.step(optimizer)
            # Updates the scale for next iteration.
            scaler.update()
            # Adjust the learning rate based on the number of iterations.
            scheduler.step()
            # Clear gradients
            optimizer.zero_grad()

        running_loss += loss.item()

        # Print training loss information
        if (iter + 1) % print_every == 0:
            print(
                f"Iteration {iter+1}/{num_iter} complete. " +
                f"Loss: {running_loss / print_every}"
            )
            running_loss = 0.0

def evaluate_loss(
    model, device,
    loss_func,
    dataloader
):
    model.eval()

    loss, count = 0, 0
    with torch.no_grad():
        for it, (encoding, labels) in enumerate(tqdm(dataloader)):
            # Converting to cuda tensors
            for k, v in encoding.items(): encoding[k] = v.to(device)
            labels = labels.to(device)

            logits = model(encoding)
            loss += loss_func(logits.squeeze(-1), labels.float()).item()
            count += 1

    mean_loss = loss / count
    return mean_loss


def train_bert(
    model,
    loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
):
    best_loss = np.Inf
    iters = []
    train_losses = []
    valid_losses = []

    scaler = GradScaler('cuda')

    for epoch in range(epochs):
        # Compute training loss
        train_loss = train_one_epoch(
            model, device, loss_func,
            scaler,  # Grad scaler
            optimizer,
            scheduler,
            train_loader,
            iters_to_accumulate,
            # print the training loss 5 times per epoch
            print_every=len(train_loader) // 5
        )
        # Compute validation loss
        valid_loss = evaluate_loss(
            model, device, loss_func,
            valid_loader
        )
        print(f"Epoch {epoch+1} complete! Validation Loss: {valid_loss}")

        if valid_loss < best_loss:
            print(f"Best validation loss improved from {best_loss} to {valid_loss}")
            best_loss = valid_loss
            # TODO: save the model

    torch.cuda.empty_cache()

In [9]:
# Parameters

bert_name = "neulab/codebert-python"

"""
other checkpoints:
- "microsoft/codebert-base"
- "huggingface/CodeBERTa-small-v1"
"""

epochs = 4
lr = 1e-5  # learning rate
wd = 1e-5  # weight decay
bs = 20    # batch size
# The gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate.
# If set to "1", you get the usual batch size
iters_to_accumulate = 2

In [10]:
# Dataset

bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)

dataset = CodePairDataset.from_csv_data(paired_dataset_url, bert_tokenizer)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_data, valid_data = random_split(dataset, [train_size, valid_size])

SHUFFLE = True
train_loader = DataLoader(train_data, batch_size=bs, shuffle=SHUFFLE)
valid_loader = DataLoader(valid_data, batch_size=bs, shuffle=SHUFFLE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# Model instance

bert_model = AutoModel.from_pretrained(bert_name).to(device)

model = CodeSimilarityClassifier(bert_model).to(device)

config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-python and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# optimizer, scheduler

optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

# The number of steps for the warmup phase.
num_warmup_steps = 0
# The total number of training steps
num_training_steps = epochs * len(train_loader)
# Necessary to take into account Gradient accumulation
num_training_steps = num_training_steps // iters_to_accumulate

scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# Train the model

# Binary cross entropy loss with sigmoid layer
loss_func = nn.BCEWithLogitsLoss()

train_bert(
    model, loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
)

<ipython-input-8-2ee25484a7a6>:94: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


  0%|          | 0/400 [00:00<?, ?it/s]

<ipython-input-8-2ee25484a7a6>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



Iteration 80/400 complete.Loss : 0.3475358720868826

Iteration 160/400 complete.Loss : 0.3500200483947992

Iteration 240/400 complete.Loss : 0.3496276654303074

Iteration 320/400 complete.Loss : 0.34664524607360364

Iteration 400/400 complete.Loss : 0.34716252982616425


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/400 [00:00<?, ?it/s]


Iteration 80/400 complete.Loss : 0.34319341033697126

Iteration 160/400 complete.Loss : 0.3405614048242569

Iteration 240/400 complete.Loss : 0.33997760340571404

Iteration 320/400 complete.Loss : 0.34074610769748687

Iteration 400/400 complete.Loss : 0.33937221989035604


  0%|          | 0/100 [00:00<?, ?it/s]


Epoch 4 complete! Validation Loss : 0.6865198141336442
Best validation loss improved from inf to 0.6865198141336442


UnboundLocalError: local variable 'loss' referenced before assignment

In [ ]:
# TODO: evaluation